<h1>Extracting region of interest from each video (person)</h1>

<h2>Imports</h2>

In [1]:
!pip install cvlib
!pip uninstall tensorflow
!pip install tensorflow==2.0.0
import numpy as np
import cv2
import glob
import cvlib as cv
from cvlib.object_detection import draw_bbox
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

     |████████████████████████████████| 10.1MB 2.8MB/s 
  Created wheel for cvlib: filename=cvlib-0.2.4-cp36-none-any.whl size=10043766 sha256=c993ba8e36ca677e9d6c3ec8b71a552ff9137b49cc75813bb5aae9083d3634bc
  Stored in directory: /root/.cache/pip/wheels/e3/2b/5e/da538fa7a4fc51ba1f3e89c57f678f9ebfd1247b9b70c80a5a
  Created wheel for progressbar: filename=progressbar-2.5-cp36-none-any.whl size=12074 sha256=8cc0fe9b6fb94f6454d97dc4f567308ab2cb63aa1a23148e78cf86121ba440f8
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built cvlib progressbar
Uninstalling tensorflow-2.2.0rc1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0rc1.dist-info/*
    /usr/local/lib/p

<h2>Helper functions</h2>

This function loads the videos from a specified path and returns them in a list.

In [0]:
def read_videos(path):
    videos = []
    videos_paths = glob.glob(path + '/*[0-9].mp4')
    for video_path in videos_paths:
      videos.append(cv2.VideoCapture(video_path))
    return videos

Next, we implement the function that applies YOLO to find the bounding boxes of the persons in one video.

This function returns a list containing the bounding boxes of each frame in one video.
If more than one bounding box are found in one frame, the bouding boxes are merged to obtain one bounding box.

In [0]:
def yolo(video):
  frames_no = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  bounding_boxes = []
  print('start')
  for frame_no in range(frames_no):
    ret, frame = video.read()
    if ret != True:
      continue
    bboxes, label, confidence = cv.detect_common_objects(frame)
    if len(bboxes) == 0:
      continue
    x_top_left = bboxes[0][0]
    y_top_left = bboxes[0][1]
    x_bottom_right = bboxes[0][2]
    y_bottom_right = bboxes[0][3]
    for i in range(1, len(bboxes)): # if more than one bbox, merge them
      x_top_left = min(x_top_left, bboxes[i][0])
      y_top_left = min(y_top_left, bboxes[i][1])
      x_bottom_right = max(x_bottom_right, bboxes[i][2])
      y_bottom_right = max(y_bottom_right, bboxes[i][3])
    bbox = [x_top_left, y_top_left, x_bottom_right, y_bottom_right]
    bounding_boxes.append(bbox)
    return bounding_boxes

This function finds the bounding box with the largest area.

In [0]:
def get_max_min_bounding_boxes(bounding_boxes):
  max_bbox = []
  max_bbox_area = -1
  min_bbox = []
  min_bbox_area = 1000000000000000
  for bbox in bounding_boxes:
    width = bbox[2] - bbox[0]
    height = bbox[3] - bbox[1]
    bbox_area = width * height
    if bbox_area > max_bbox_area:
      max_bbox_area = bbox_area
      max_bbox = bbox
    if bbox_area < min_bbox_area:
      min_bbox_area = bbox_area
      min_bbox = bbox
  return max_bbox, max_bbox_area, min_bbox, min_bbox_area

This function crops one video according to the input bounding_box. It also resizes the video to the required size.

In [0]:
def crop_video(video, bounding_box):
    x_top_left = int(max(0, bounding_box[0]))
    y_top_left = int(max(0, bounding_box[1]))
    x_bottom_right = int(min(video.get(3) - 1, bounding_box[2])) # video.get(3) = width
    y_bottom_right = int(min(video.get(4) - 1, bounding_box[3])) # video.get(4) = height
    frames_no = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    cropped_frames = []
    for frame_no in range(frames_no):
        ret, frame = video.read()
        if ret != True:
          continue
        cropped_frame = frame[y_top_left:y_bottom_right, x_top_left:x_bottom_right]
        cropped_frames.append(cropped_frame)
    return cropped_frames

In [0]:
def find_min_frames(videos):
  min_frames = 10000000
  for video in videos:
    frames_no = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    if frames_no < min_frames:
      min_frames = frames_no
  return min_frames

<h2>Reading videos</h2>

In [0]:
def read():
  path = '/content/gdrive/My Drive/Team\'s Drive/Graduation Project/Project/Dataset/Clips'
  truthful_videos = read_videos(path + '/Truthful')
  deceptive_videos = read_videos(path + '/Deceptive')
  return truthful_videos, deceptive_videos

<h3>Applying YOLO to get the bounding boxes per frame per video.</h3>

Example:
To access the bounding boxes of the second frame in the first truthful video: truthful_boundingboxes[0][1] (list)

<h3>Finding the maximum possible bounding box for each video.</h3>

In [0]:
truthful_videos, deceptive_videos = read()
max_bboxes_true = []
max_bboxes_lie = []
for i in range(len(truthful_videos)):
  vid = truthful_videos[i]
  bboxes = yolo(vid)
  max_bbox, max_area, min_bbox, min_area = get_max_min_bounding_boxes(bboxes)
  max_bboxes_true.append(max_bbox)
  #with open('true_bboxes/true' + str(i) + '.txt', 'w') as f:
  # f.write(np.array2string(np.asarray(max_bbox + min_bbox)))

for i in range(len(deceptive_videos)):
  vid = deceptive_videos[i]
  bboxes = yolo(vid)
  max_bbox, max_area, min_bbox, min_area = get_max_min_bounding_boxes(bboxes)
  max_bboxes_lie.append(max_bbox)
  #with open('lie_bboxes/lie' + str(i) + '.txt', 'w') as f:
  #  f.write(np.array2string(np.asarray(max_bbox + min_bbox)))


In [0]:
#!zip -r /content/true_bboxes.zip /content/true_bboxes
#!zip -r /content/lie_bboxes.zip /content/lie_bboxes

In [0]:
import os
truthful_videos, deceptive_videos = read()
min_true = find_min_frames(truthful_videos)
min_lie = find_min_frames(deceptive_videos)
minn = min(min_true, min_lie)
truthful_videos, deceptive_videos = read()
os.mkdir('Truthful')
for i in range(len(truthful_videos)):
    video = truthful_videos[i]
    cropped_frames = crop_video(video, max_bboxes_true[i])
    folder_name = 'Truthful/trial_truth_' + str(i + 1).zfill(3)
    os.mkdir(folder_name)
    if len(cropped_frames) <= minn:
      for j in range(len(cropped_frames)):
        num = str(j).zfill(3)
        cv2.imwrite(folder_name + '/' + num + '.jpg', cropped_frames[j])
    else:
      sub_vids = len(cropped_frames) // minn
      for k in range(sub_vids):
        subfolder_name = str(k).zfill(0)
        new_dir = folder_name + '/' + subfolder_name 
        os.mkdir(new_dir)
        for j in range(minn):
          index = j + k * minn
          num = str(index).zfill(3)
          cv2.imwrite(new_dir + '/' + num + '.jpg', cropped_frames[index])
      remainder = len(cropped_frames) % minn
      k += 1
      if remainder > 0.5 * minn:
        missing = minn - remainder
        start_idx = len(cropped_frames) - remainder - missing
        subfolder_name = str(k).zfill(0)
        new_dir = folder_name + '/' + subfolder_name 
        os.mkdir(new_dir)
        while start_idx < len(cropped_frames):
          num = str(start_idx).zfill(3)
          cv2.imwrite(new_dir + '/' + num + '.jpg', cropped_frames[start_idx])
          start_idx += 1
        k += 1

In [0]:
truthful_videos, deceptive_videos = read()
os.mkdir('Deceptive')
for i in range(len(deceptive_videos)):
    video = deceptive_videos[i]
    cropped_frames = crop_video(video, max_bboxes_lie[i])
    folder_name = 'Deceptive/trial_lie_' + str(i + 1).zfill(3)
    os.mkdir(folder_name)
    if len(cropped_frames) <= minn:
      for j in range(len(cropped_frames)):
        num = str(j).zfill(3)
        cv2.imwrite(folder_name + '/' + num + '.jpg', cropped_frames[j])
    else:
      sub_vids = len(cropped_frames) // minn
      for k in range(sub_vids):
        subfolder_name = str(k).zfill(0)
        new_dir = folder_name + '/' + subfolder_name 
        os.mkdir(new_dir)
        for j in range(minn):
          index = j + k * minn
          num = str(index).zfill(3)
          cv2.imwrite(new_dir + '/' + num + '.jpg', cropped_frames[index])
      remainder = len(cropped_frames) % minn
      k += 1
      if remainder > 0.5 * minn:
        missing = minn - remainder
        start_idx = len(cropped_frames) - remainder - missing
        subfolder_name = str(k).zfill(0)
        new_dir = folder_name + '/' + subfolder_name 
        os.mkdir(new_dir)
        while start_idx < len(cropped_frames):
          num = str(start_idx).zfill(3)
          cv2.imwrite(new_dir + '/' + num + '.jpg', cropped_frames[start_idx])
          start_idx += 1
        k += 1

In [0]:
!cp -r /content/Truthful /content/gdrive/"My Drive"/"Team's Drive"/"Graduation Project"/Project/Dataset/Cropped
!cp -r /content/Deceptive /content/gdrive/"My Drive"/"Team's Drive"/"Graduation Project"/Project/Dataset/Cropped